In [39]:
import os
import boto3

session = boto3.Session(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_KEY"),
)

client = session.client("bedrock-runtime", "us-east-1")


In [42]:
from langchain.chat_models import BedrockChat
from langchain.prompts import ChatPromptTemplate

chat = BedrockChat(
    client=client,
    model_id="anthropic.claude-v2:1",
    model_kwargs={
        "temperature": 0.1,
    },
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            "Translate this sentence from {lang_a} to {lang_b}: {sentence}",
        ),
    ]
)

chain = prompt | chat

chain.invoke(
    {
        "lang_a": "English",
        "lang_b": "Icelandic",
        "sentence": "I love amazon!",
    }
)


ValueError: Error raised by bedrock service: Unable to locate credentials